<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
 <h2>Mentoría: Clasificación diagnóstica de mamografías </h2>
</center>

<h3> Práctico III y IV -  Aprendizaje Supervisado <h3>
<h4>Integrantes: Mario Agustín Sgró, Lucía Benítez y Carolina Díaz <h4>

**Objetivo y alcance:** 
  Conocimiento y práctica sobre técnicas básicas de aprendizaje automático discriminando por tipo de variable a predecir. Estudiar aumentación de datos polinomial asociada a cambio de modelo. Continuación del práctico anterior. Pueden entregarse juntos presentando únicamente este práctico, pues incluye lo pedido en el anterior.

**Método:** 
Regresión lineal y polinomial, clasificación Regulación.


**Estructura del informe:** 
Presentar en un archivo jupyter notebook con la resolución detallada de las siguientes consignas:

1- Separar la base en dos conjuntos: Train (con el 70 % de los datos) y Test con el 30%) 
 
2- Elegir 2 variables/características/features numéricas de la base de datos, una proveniente del grupo de las Haralick y otra asociada a la Dimensión Fractal:

            a) Considerando una como regresora y la otra como objetivo o variable/característica a predecir, realizar una regresión lineal y varias polinomiales (hasta grado 5) para evaluar el grado del polinomio que mejor se ajusta a las predicciones (Kernel trick). Realizar la búsqueda utilizando validación cruzada  en el conjunto Train, con RMSE o análoga como medida de calidad de ajuste.
            b) Agregar un parámetro de regulación, convirtiendo a esta búsqueda en búsqueda por grilla.
            c) Una vez definidos los hiperparámetros (encontrados en la búsqueda del item anterior. Evaluar el modelo ajustado utilizando el conjunto Test, con la medida (o las medidas) que crea adecuada.

3- Definir una nueva variable categórica (binaria) con clase 1 como clase Ay B y clase 2 como clase Cy D de densidad mamaria. Y reducir la base considerando además sólo dos de todas las variables numéricas, es decir, la nueva base tendrá sólo tres variables. Pero sigue estando dividida en los conjuntos Train y Test.
Implementar el algoritmo de perceptrón en el conjunto Train para clasificación binaria: clase 1 vs clase 2. 
Evaluar con la medidas que crea adecuadas y graficar los resultados en tres instancias de aprendizaje (con un tercio de la base, con 2 tercios y con la base Train). 
Evaluar cada instancia en el conjunto Test y comparar los resultados con los resultados sobre Train.
Comentar si se presenta sobre ajuste, sesgo o ninguno.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sns
#import chardet
import string
plt.rcParams['figure.figsize'] = (5, 5)

In [ ]:
if sns.__version__ != '0.9.0':
    print('Atención! utilizamos seaborn versión 0.9.0')

In [ ]:
pd.options.display.max_columns = 200
#pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 1000

# Carga del DataSet

In [ ]:
#data = pd.read_csv('datos/Datos_Mamografias.csv',sep=',',index_col=['Imagen'])

Por lo analizado en el práctico anterior, sabemos que la base de datos posee filas con datos faltantes, por lo que en lo siguiente, ocuparemos una base de datos ya limpia, con sus filas sin datos eliminadas y con la imputación de datos faltantes por medio de vecinos mas cercanos (knn):

In [ ]:
dataframe = pd.read_csv('datos/KnnFilled_data.csv',sep=',',index_col=['Imagen'])

## Descripción de la base de datos:

In [ ]:
dataframe.head(3)

In [ ]:
print("El data set consiste de ",dataframe.shape[0]," imagenes, con ",dataframe.shape[1]," registros cada una.")

In [ ]:
dataframe['Dcm_6'] = dataframe['Dcm_6'].astype('category')
dataframe['Dcm_13'] = dataframe['Dcm_13'].astype('category')
dataframe['Dcm_11'] = dataframe['Dcm_11'].astype('category')
dataframe['Dcm_21'] = dataframe['Dcm_21'].astype('category')
dataframe['Dcm_22'] = dataframe['Dcm_22'].astype('category')

Para el presente práctico se propone hacer la división de los _targets_ en dos grandes clases, una que contiene a los datos con la variable "ACR" igual a _a_ o _b_ y la otra con los datos con dicha variable igual a _c_ o _d_. Por ello creamos una nueva variable target:

In [ ]:
dataframe["Class"] = dataframe["ACR"].copy()

dataframe["Class"].replace({"a":0,"b":0,"c":1,"d":1},inplace=True)
dataframe['Class'] = dataframe['Class'].astype('category')

## Tipos de Variables

In [ ]:
CatFeat = [x for x in dataframe.columns if dataframe[x].dtype == 'object']
NumFeat = [x for x in dataframe.columns if dataframe[x].dtype == 'float64']
IntFeat = [x for x in dataframe.columns if dataframe[x].dtype == 'int64']

print('Cantidad' + str('\n')+
      '   variables numéricas     '+ str(len(NumFeat)) + str('\n') +
      '   variables enteras     '+ str(len(IntFeat)) + str('\n')
      + '   variables discretas  ' + str(len(CatFeat)))

In [ ]:
for cat in CatFeat:
    dataframe[cat] = dataframe[cat].astype('category')

In [ ]:
CatFeat = [x for x in dataframe.columns if dataframe[x].dtype.name == 'category']
NumFeat = [x for x in dataframe.columns if dataframe[x].dtype.name != 'category']

print('Cantidad' + str('\n')+
      '   variables numéricas     '+ str(len(NumFeat)) + str('\n')
      + '   variables discretas  ' + str(len(CatFeat)))

Transformamos las variables categóricas con palabras en numéricas

In [ ]:
data = dataframe.copy()
data.shape

In [ ]:
for cat in CatFeat:
    data[cat] = data[cat].cat.codes

## Classification

1) Dividimos la muestra en un conjunto de training y uno de test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["ACR","Class"],axis=1),data.Class,
                                                    test_size=0.3,
                                                    random_state=167)

In [ ]:
print(X_train.shape,X_test.shape)

In [ ]:
feature_map = {feature: idx for idx, feature in enumerate(X_train.columns)}

Checkeamos el desbalance de clases 0 y 1

In [ ]:
bal = np.bincount(y_test)
print(bal[1]/bal[0])

Observamos que las clases están bien balanceadas

Cargamos algunas herramientas que utilizaremos luego

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, mean_squared_error

In [ ]:
from time import time
from sklearn.model_selection import GridSearchCV

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

np.set_printoptions(precision=2)

## 2) Regresión lineal y polinomial:

Tomamos 2 variables numéricas de la base de datos, una de las Haralick (Haralick_46)y otra correspondiente a la Dimensión Fractal (DF16):

In [ ]:
sns.pairplot(dataframe,hue='ACR',x_vars=['Haralick_46','DF16'],y_vars=['Haralick_46','DF16'])
sns.despine()

a) Tomamos a la característica Haralick como regresora y la DF como variable objetivo: 

In [ ]:
x = dataframe.Haralick_46.values
y = dataframe.DF16.values

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

In [ ]:
plt.figure(figsize=(20, 4))
grados = np.arange(1,6)
for i in range(len(grados)):
    ax = plt.subplot(1, len(grados), i + 1)
    
    polynomial_features = PolynomialFeatures(degree=grados[i],include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(x[:, np.newaxis], y)

    scores = cross_val_score(pipeline, x[:, np.newaxis],y,scoring="neg_mean_squared_error",cv=10)

    X_test = np.linspace(x.min(), x.max(), 100)
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Modelo")
    plt.scatter(x, y, edgecolor='b', s=20, label="Datos")
    plt.xlabel(dataframe.Haralick_46.name)
    plt.ylabel(dataframe.DF16.name)
    plt.legend(loc="best")
    plt.title("Grado {}\nMSE = {:.2e}(+/- {:.2e})".format(grados[i], -scores.mean(), scores.std()))
plt.show()

Podemos notar que la relación entre ellas no es lineal y que el polinomio que mejor se ajusta es el de 3er grado. 

Con regularización:

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV, train_test_split

In [ ]:
polynomial_degree = 3

for alpha in np.logspace(-6,0.0,10):

    poly_features = PolynomialFeatures(polynomial_degree)
    poly_features.fit(x[:, np.newaxis])
    X_poly_train = poly_features.transform(x[:, np.newaxis])

    model = Ridge(alpha=alpha,normalize=True,fit_intercept=True)
    model.fit(X_poly_train, y)

    print('valor de alpha%.2e: Media del error cuadrado para entrenamiento: %.2f' % (alpha,mean_squared_error(y, model.predict(X_poly_train))))
   

In [ ]:
polynomial_degree = 3

for alpha in np.logspace(-6,0.0,10):

    poly_features = PolynomialFeatures(polynomial_degree)
    poly_features.fit(x[:, np.newaxis])
    X_poly_train = poly_features.transform(x[:, np.newaxis])

    model = Lasso(alpha=alpha,normalize=True,fit_intercept=True)
    model.fit(X_poly_train, y)

    print('%.2e: Media del error cuadrado para entrenamiento: %.2f' % (alpha,mean_squared_error(y, model.predict(X_poly_train))))

En ambos casos, tanto en Ridge como en Lasso, alpha=2.15e-05 puede tomarse como parámetro de regularización antes de que aumente el error cuadrático medio



Validación cruzada


In [ ]:
polynomial_degree = 3
alpha = 2.15e-5

poly_features = PolynomialFeatures(polynomial_degree)
poly_features.fit(x[:, np.newaxis])
X_poly_train = poly_features.transform(x[:, np.newaxis])

model = Lasso(alpha=alpha,normalize=True,fit_intercept=True)
model.fit(X_poly_train, y)

X_test = np.linspace(x.min(), x.max(), 100).reshape(-1,1)
X_poly_test = poly_features.transform(X_test)

scores = cross_val_score(model, X_poly_train,y,scoring="neg_mean_squared_error",cv=10)

plt.plot(X_test, model.predict(X_poly_test), label="Model")
plt.scatter(x, y, edgecolor='b', s=20, label="Samples")
plt.xlabel(dataframe.Haralick_46.name)
plt.ylabel(dataframe.DF16.name)
plt.legend(loc="best")
plt.title("Grado {}\nMSE = {:.1f}".format(polynomial_degree, mean_squared_error(y, model.predict(X_poly_train))))
plt.show()

### 3. Implementar el algoritmo de perceptrón para clasificación binaria: consideranco clase 1 como clase Ay B y a clase 2 como clase Cy D de densidad mamaria. 

## Perceptrón

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["ACR","Class"],axis=1),data.Class,
                                                    test_size=0.3,
                                                    random_state=167)

In [ ]:
print(X_train.shape,X_test.shape)

Para el algoritmo de perceptrón debemos tener normalizados los datos

In [ ]:
X_train_norm = X_train.copy()
X_test_norm = X_test.copy()

In [ ]:
Min = X_train_norm.min()
Max = X_train_norm.max()
Mean = X_train_norm.mean()

In [ ]:
print(X_train_norm.shape,X_test_norm.shape)

In [ ]:
X_train_norm = (X_train_norm - Mean)/(Max - Min)
X_test_norm = (X_test_norm - Mean)/(Max - Min)

### Utilizando todos los datos

In [ ]:
from sklearn.linear_model import Perceptron

#### Consideramos los valores por defecto del estimador

In [ ]:
clf = Perceptron(tol=1.e-7,random_state=167,fit_intercept=True,max_iter=10000,validation_fraction=0.3)
clf.fit(X_train_norm,y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(y_test, y_pred, classes = np.array([0,1]), 
                      normalize=False,
                      title='Confusion matrix')
sns.despine()

#### Buscamos los mejores parámetros

In [ ]:
clf.get_params().keys()

In [ ]:
clf = Perceptron(random_state=42,fit_intercept=True,max_iter=10000,tol=1.0e-7,validation_fraction=0.3)

#presort True o False da lo mismo

alphas = np.logspace(-8,0,9)
etas = np.logspace(-8,0,9)


param_grid = {"alpha":alphas,
              "eta0": etas,
              "penalty": ("l1","l2",None,"elasticnet")
             }

grid = GridSearchCV(clf,param_grid=param_grid,cv=5,scoring="f1",iid=False,n_jobs=8)
start = time()
grid.fit(X_train_norm, y_train)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid.cv_results_['params'])))
report(grid.cv_results_)

In [ ]:
grid.score(X_test_norm,y_test)

In [ ]:
y_pred = grid.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(y_test, y_pred, classes = np.array([0,1,2]), 
                      normalize=False,
                      title='Confusion matrix')
sns.despine()

## Selección de Características

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.feature_selection import SelectFromModel

In [ ]:
clf = Perceptron(random_state=42,alpha=0.01,penalty='l1',fit_intercept=True,max_iter=10000,tol=1.0e-7)
clf = clf.fit(X_train_norm,y_train)
model = SelectFromModel(clf,prefit=True,threshold=-np.inf,max_features=20)
X_new = model.transform(X_train_norm)
print(X_new.shape)

In [ ]:
indices = model.get_support(indices=True)

In [ ]:
X_train_norm.columns[indices]

Aqui no nos seleccionó ninguna Haralick :(.. Probaremos algo más a fuerza bruta

### Fuerza bruta

In [ ]:
def best_features(feature1,feature2,penalty='l1',alpha = 0.01,max_iter = 10000):
    X_train_feature = X_train_norm.loc[:, [feature1, feature2]]
    X_test_feature = X_test_norm.loc[:, [feature1, feature2]]
    
    model = Perceptron(penalty=penalty, alpha=alpha, max_iter=max_iter,tol=1.e-7)
    model.fit(X_train_feature, y_train)

    return(accuracy_score(y_train, model.predict(X_train_feature)),accuracy_score(y_test, model.predict(X_test_feature)))    

_myf1 = ''
_myf2 = ''

for _p in ['l1','l2','elasticnet']:
    score = 0.0
    for feature1 in NumFeat:
        for feature2 in NumFeat:
            if feature1 == feature2:
                continue
    
            _x, _y = best_features(feature1,feature2,penalty=_p,alpha=10.0)
            if _x > score:
                _myf1 = feature1
                _myf2 = feature2
                score = _x
    print(_p,"feature1: ",_myf1,"feature2: ",_myf2,score)

## Perceptrón sobre características seleccionadas

In [ ]:
from sklearn.datasets import make_classification
from ml.visualization import classifier_boundary
from matplotlib.colors import ListedColormap
from utils import plot_decision_boundary

In [ ]:
# Seleccionamos dos atributo de los listados en el apartado anterior, uno para el eje x y otro para el eje y
x_feature = 'Haralick_48'
y_feature = 'DFb31'

X_train_feature = X_train_norm.loc[:, [x_feature, y_feature]]
X_test_feature = X_test_norm.loc[:, [x_feature, y_feature]]

In [ ]:
model = Perceptron(max_iter=10000,tol=1.e-7,alpha=0.01, eta0=1e-06, penalty='l2')
model.fit(X_train_feature, y_train)
model.score(X_test_feature,y_test)

In [ ]:
plt.figure(figsize=(14, 5), dpi=80, facecolor='w', edgecolor='k')

xx, yy, Z = classifier_boundary(np.r_[X_train_feature, X_test_feature], model,h=0.01)


cmap_dots = ListedColormap(['tomato', 'dodgerblue'])
cmap_back = ListedColormap(['lightcoral', 'skyblue'])

# Conjunto de entrenamiento
plt.subplot(1, 2, 1)
plt.pcolormesh(xx, yy, Z, cmap=cmap_back)
plt.scatter(X_train_feature.iloc[:, 0], X_train_feature.iloc[:, 1], c=y_train, cmap=cmap_dots, edgecolor=None, s=1)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("Conjunto de Entrenamiento")

# Conjunto de validación
plt.subplot(1, 2, 2)
plt.pcolormesh(xx, yy, Z, cmap=cmap_back)
plt.scatter(X_test_feature.iloc[:, 0], X_test_feature.iloc[:, 1], c=y_test, cmap=cmap_dots, edgecolor=None, s=1)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("Conjunto de Validación")

plt.show()

#### Buscamos mejores parámetros

In [ ]:
clf = Perceptron(random_state=42,fit_intercept=True,max_iter=10000,tol=1.0e-7,validation_fraction=0.3)

#presort True o False da lo mismo

alphas = np.logspace(-12,0,13)
etas = np.logspace(-12,0,13)


param_grid = {"alpha":alphas,
              "eta0": etas,
              "penalty": ("l1","l2",None,"elasticnet")
             }

grid = GridSearchCV(clf, param_grid=param_grid,cv=5,scoring="f1", iid=False,n_jobs=4)
start = time()
grid.fit(X_train_feature, y_train)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid.cv_results_['params'])))
report(grid.cv_results_)

Se obtiene que los mejores párametros para el algoritmo de perceptrón son:

    alpha = 1.e-5
    eta = 1.e-12
    penalty = l1

### 4. Evaluar y graficar los resultados en tres instancias de aprendizaje utilizando dos de todas las variables numéricas. Con imax, se define distintas porciones de train.

In [ ]:
fig, axs = plt.subplots(3,2,figsize=(10,15))

cmap_dots = ListedColormap(['tomato', 'dodgerblue'])
cmap_back = ListedColormap(['lightcoral', 'skyblue'])
  
for i in range(3):        

    _imax = X_train_feature.shape[0]//3*(i+1)
    print(_imax)
    
    _X = X_train_feature.iloc[0:_imax,:]
    _Y = y_train.iloc[0:_imax]

    model = Perceptron(max_iter=10000,tol=1.0e-7,alpha=1.0e-5, eta0=1.0e-12, penalty='l1',random_state=617)
    model.fit(_X, _Y)
      
    print(model.n_iter_,model.coef_,model.score(X_test_feature,y_test))
        
    xx, yy, Z = classifier_boundary(np.r_[_X, X_test_feature], model,h=0.01)

    # Conjunto de entrenamiento
    axs[i,0].pcolormesh(xx, yy, Z, cmap=cmap_back)
    axs[i,0].scatter(_X.iloc[:, 0], _X.iloc[:, 1], c=_Y, cmap=cmap_dots, edgecolor=None, s=1)
    axs[i,0].set_xlim(-0.2,0.2)
    axs[i,0].set_ylim(-0.2,0.2)
        
    if i == 0:
        axs[i,0].set_title("Conjunto de Entrenamiento")

    # Conjunto de validación
    axs[i,1].pcolormesh(xx, yy, Z, cmap=cmap_back)
    axs[i,1].scatter(X_test_feature.iloc[:, 0], X_test_feature.iloc[:, 1], c=y_test, cmap=cmap_dots, edgecolor=None, s=1)
    axs[i,1].set_xlim(-0.2,0.2)
    axs[i,1].set_ylim(-0.2,0.2)
        
    if i == 0:
        axs[i,1].set_title("Conjunto de Validación")
    
plt.show()

Probando otros clasificadores: 

## Decission Tree Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier as DTC

In [ ]:
clf = DTC(random_state=167)
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(y_test, y_pred, classes = np.array([0,1,2]), 
                      normalize=False,
                      title='Confusion matrix')

In [ ]:
clf.get_params().keys()

In [ ]:
clf = DTC(random_state=42,splitter="best",presort=True,criterion="entropy")

#presort True o False da lo mismo

param_grid = {#"criterion":("gini","entropy"),
              "max_features": (50,55,60,65,70),
              "min_samples_split": (2,3,4)
             }

grid = GridSearchCV(clf, param_grid=param_grid, cv=5,scoring="f1", iid=False,n_jobs=4)
start = time()
grid.fit(X_train, y_train)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid.cv_results_['params'])))
report(grid.cv_results_)

In [ ]:
grid.score(X_test,y_test)

In [ ]:
y_pred = grid.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(y_test, y_pred, classes = np.array([0,1,2]), 
                      normalize=False,
                      title='Confusion matrix')

## Bagging Classification

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
bagging = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)

In [ ]:
bagging.fit(X_train,y_train)

In [ ]:
bagging.score(X_test,y_test)

In [ ]:
y_pred = bagging.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(y_test, y_pred, classes = np.array([0,1,2]), 
                      normalize=False,
                      title='Confusion matrix')
sns.despine()

Obtenemos un score de .75, aun sin optimizar. Veamos si podemos optimizar un poco

In [ ]:
bagging.get_params().keys()

In [ ]:
clf = BaggingClassifier(KNeighborsClassifier(), random_state=42)

param_grid = {"max_samples": (0.5,0.8),
              "max_features": (0.01,0.05),
              "bootstrap": (True, False),
              "n_estimators": (50,60)
             }

grid = GridSearchCV(clf, param_grid=param_grid, cv=5, iid=False,n_jobs=4)
start = time()
grid.fit(X_train, y_train)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid.cv_results_['params'])))
report(grid.cv_results_)

In [ ]:
y_pred = grid.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(y_test, y_pred, classes = np.array([0,1,2]), 
                      normalize=False,
                      title='Confusion matrix')
sns.despine()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC

In [ ]:
clf = RFC(random_state=169)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(y_test, y_pred, classes = np.array([0,1,2]), 
                      normalize=False,
                      title='Confusion matrix')
sns.despine()

Obtenemos un score de .82, aun sin optimizar. Veamos si podemos optimizar un poco

In [ ]:
clf.get_params().keys()

In [ ]:
clf = RFC(random_state=42)

param_grid = {
              "max_features": ("sqrt",None),
              "bootstrap": (True, False),
              "n_estimators": (60,100)
             }

grid = GridSearchCV(clf, param_grid=param_grid, cv=5, iid=False,n_jobs=4,scoring="f1")
start = time()
grid.fit(X_train, y_train)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid.cv_results_['params'])))
report(grid.cv_results_)

In [ ]:
y_pred = grid.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(y_test, y_pred, classes = np.array([0,1,2]), 
                      normalize=False,
                      title='Confusion matrix')
sns.despine()